# CONSTRUIR BD EN NEO4J

In [1]:
from uuid import uuid4
from pydantic import BaseModel, Field
from typing import List
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_core.prompts.prompt import PromptTemplate



class Persona(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid4()))
    nombre: str
    edad: int
    ciudad: str


class Coche(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid4()))
    marca: str
    modelo: str
    anio: int

class Entidades(BaseModel):
    """Contenedor de todas las entidades"""
    personas: List[Persona] = Field(default_factory=list)
    coches: List[Coche] = Field(default_factory=list)

#relaciones
class Viaje(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid4()))
    persona: Persona
    coche: Coche
    descripcion_relacion: str

class Relaciones(BaseModel):
    """Contenedor de todos los pares relacionados"""
    relaciones: List[Viaje] = Field(default_factory=list)

class OutputSchema(BaseModel):
    entidades: Entidades
    relaciones: Relaciones

In [3]:
# Personas
juan = Persona(nombre="Juan", edad=30, ciudad="Madrid")
ana = Persona(nombre="Ana", edad=25, ciudad="Barcelona")
luis = Persona(nombre="Luis", edad=40, ciudad="Valencia")

# Coches
toyota = Coche(marca="Toyota", modelo="Corolla", anio=2018)
ford   = Coche(marca="Ford", modelo="Focus", anio=2020)
tesla  = Coche(marca="Tesla", modelo="Model 3", anio=2022)

# Viajes
viaje1 = Viaje(persona=juan, coche=toyota,descripcion_relacion="Fue su primer coche")
viaje2 = Viaje(persona=ana, coche=tesla,  descripcion_relacion="Viajó con su madre para ver el nuevo modelo")
viaje3 = Viaje(persona=luis, coche=ford,  descripcion_relacion="Tuvo un accidente y no pudo viajar")


In [4]:
entidades = Entidades(personas=[juan, ana, luis], coches=[toyota, ford, tesla])
relaciones = Relaciones(relaciones=[viaje1, viaje2, viaje3])
output_schema = OutputSchema(entidades=entidades, relaciones=relaciones)

In [5]:

# Conectar a Neo4j
graph = Neo4jGraph(url="bolt://localhost:50001", username="neo4j", password="frandovi", database = "prueba-db")



In [6]:
print(graph.schema)


Node properties:
Persona {id: STRING, edad: INTEGER, ciudad: STRING, nombre: STRING}
Coche {id: STRING, anio: INTEGER, marca: STRING, modelo: STRING}
Relationship properties:
VIAJO_EN {descripcion_relacion: STRING}
The relationships:
(:Persona)-[:VIAJO_EN]->(:Coche)


In [7]:
# Eliminar todos los nodos y relaciones
graph.query("MATCH (n) DETACH DELETE n")

# eliminar constraints/índices si los creaste
graph.query("DROP CONSTRAINT persona_id IF EXISTS")
graph.query("DROP CONSTRAINT coche_id IF EXISTS")
graph.query("DROP CONSTRAINT viaje_rel_id IF EXISTS")



[]

In [8]:
output_schema.entidades.personas

[Persona(id='bdb38422-17cd-43df-a910-f041624aa134', nombre='Juan', edad=30, ciudad='Madrid'),
 Persona(id='e3bdcf26-a0b5-4af0-8c2f-402dc1b24271', nombre='Ana', edad=25, ciudad='Barcelona'),
 Persona(id='f9643973-b84b-4c49-8705-92acd488047c', nombre='Luis', edad=40, ciudad='Valencia')]

In [9]:

# constraints
graph.query("""
CREATE CONSTRAINT persona_id IF NOT EXISTS
FOR (p:Persona) REQUIRE p.id IS UNIQUE;
""")
graph.query("""
CREATE CONSTRAINT coche_id IF NOT EXISTS
FOR (c:Coche) REQUIRE c.id IS UNIQUE;
""")
graph.query("""
CREATE CONSTRAINT viaje_rel_id IF NOT EXISTS
FOR ()-[v:VIAJO_EN]-() REQUIRE v.id IS UNIQUE;
""")

#ENTIDADES
##personas
for persona in output_schema.entidades.personas:
    graph.query(
        """
        MERGE (p:Persona {id: $id})
        SET p.nombre = $nombre, p.edad = $edad, p.ciudad = $ciudad
        """,
        params = {
            "id": persona.id,
            "nombre": persona.nombre,
            "edad": persona.edad,
            "ciudad": persona.ciudad
        }

    )

##coches
for coche in output_schema.entidades.coches:
    graph.query(
        """
        MERGE (c:Coche {id: $id})
        SET c.marca = $marca, c.modelo = $modelo, c.anio = $anio
        """,
        params = {
            "id": coche.id,
            "marca": coche.marca,
            "modelo": coche.modelo,
            "anio": coche.anio
        }
    )

for viaje in output_schema.relaciones.relaciones:
    graph.query(
        """
        MATCH (p:Persona {id: $persona_id}), (c:Coche {id: $coche_id})
        MERGE (p)-[r:VIAJO_EN {descripcion_relacion: $descripcion_relacion}]->(c)
        """,
        params={
            "persona_id": viaje.persona.id,
            "coche_id": viaje.coche.id,
            "descripcion_relacion": viaje.descripcion_relacion
        }
    )

# CONSULTAS A LA BD EN LENGUAJE NATURAL

## CONVERTIR LENGUAJE NATURAL A CYPHER
- Cadena que convierte una consulta de lenguaje natural a lenguaje cypher

- Para administrar permisos, se necesita versión Enterprise, la gratuita es community

In [43]:


# graph.query("""
# CREATE USER lector SET PASSWORD 'tu_password' CHANGE NOT REQUIRED;
# GRANT MATCH {*} ON GRAPH * TO lector;
# DENY WRITE {*} ON GRAPH * TO lector;
# """)

- Esto es lo que ocurre internamente

In [10]:
CYPHER_GENERATION_TEMPLATE = """
Schema:
{schema}

The question is:
{question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)


In [ ]:
print(graph.schema)



Node properties:
Persona {nombre: STRING, edad: INTEGER, ciudad: STRING, id: STRING}
Coche {modelo: STRING, anio: INTEGER, marca: STRING, id: STRING}
Relationship properties:
VIAJO_EN {descripcion_relacion: STRING}
The relationships:
(:Persona)-[:VIAJO_EN]->(:Coche)


In [11]:
print(graph.schema)



Node properties:
Persona {id: STRING, edad: INTEGER, ciudad: STRING, nombre: STRING}
Coche {id: STRING, anio: INTEGER, marca: STRING, modelo: STRING}
Relationship properties:
VIAJO_EN {descripcion_relacion: STRING}
The relationships:
(:Persona)-[:VIAJO_EN]->(:Coche)


In [12]:
prompt_text = CYPHER_GENERATION_PROMPT.format(
    schema=graph.schema, 
    question="Quién viajó en un tesla?"
)

print(prompt_text)



Schema:
Node properties:
Persona {id: STRING, edad: INTEGER, ciudad: STRING, nombre: STRING}
Coche {id: STRING, anio: INTEGER, marca: STRING, modelo: STRING}
Relationship properties:
VIAJO_EN {descripcion_relacion: STRING}
The relationships:
(:Persona)-[:VIAJO_EN]->(:Coche)

The question is:
Quién viajó en un tesla?



In [32]:
CYPHER_GENERATION_TEMPLATE = """Tarea: Genera una sentencia Cypher para consultar una base de datos de grafos.
Instrucciones:
- Usa únicamente los tipos de nodos, relaciones y propiedades que aparecen en el esquema.
- NO inventes etiquetas, relaciones o propiedades.
- **En el RETURN usa SIEMPRE alias fijos: p.nombre AS nombre, c.marca AS marca, c.modelo AS modelo, c.anio AS anio.**

Esquema:
{schema}

Nota: 
- No incluyas explicaciones ni disculpas.
- Devuelve únicamente la sentencia Cypher generada.

Ejemplos:
# ¿En qué coches viajó Juan?
MATCH (p:Persona {{nombre:"Juan"}})-[:VIAJO_EN]->(c:Coche)
RETURN c.marca AS marca, c.modelo AS modelo

# ¿Cuántas personas viajaron en un coche de marca Toyota?
MATCH (p:Persona)-[:VIAJO_EN]->(c:Coche {{marca:"Toyota"}})
RETURN count(p) AS numeroDePersonas

La pregunta es:
{question}"""




CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema","question"],
    template=CYPHER_GENERATION_TEMPLATE
)

QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Eres un asistente que responde EXCLUSIVAMENTE con el contexto (lista de diccionarios en JSON).\n"
        "Reglas:\n"
        "1) Si el contexto está vacío (lista vacía), responde EXACTAMENTE: 'No se encontraron resultados'.\n"
        "2) Si el contexto NO está vacío, NUNCA respondas 'No se encontraron resultados'. Debes responder con los datos.\n"
        "3) Si alguna fila tiene una clave 'nombre' o 'nombrePersona', devuelve los valores únicos de esas claves, separados por comas y sin texto adicional.\n"
        "4) Si la pregunta pide un conteo y hay una clave 'numeroDePersonas' o 'count', devuelve solo el número.\n"
        "5) Si no aplican las reglas anteriores, devuelve un resumen mínimo usando los campos disponibles del contexto.\n\n"
        "Contexto:\n{context}\n\n"
        "Pregunta:\n{question}\n\n"
        "Respuesta breve:"
    ),
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,                            # usa el schema directamente del grafo
    cypher_prompt=CYPHER_GENERATION_PROMPT, # usa info del grafo como contexto (nodos, relaciones)
    qa_prompt=QA_PROMPT,                    # instrucciones para generar la respuesta
    verbose=True,
    allow_dangerous_requests=True,
)


In [34]:
print(chain.invoke({"query":"Ana tuvo un tesla?"}))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Persona {nombre:"Ana"})-[:VIAJO_EN]->(c:Coche {marca:"Tesla"})
RETURN p.nombre AS nombre, c.marca AS marca, c.modelo AS modelo, c.anio AS anio
Full Context:
[{'nombre': 'Ana', 'marca': 'Tesla', 'modelo': 'Model 3', 'anio': 2022}]

> Finished chain.
{'query': 'Ana tuvo un tesla?', 'result': 'Sí.'}


## USAR EMBEDDINGS
- Se almacena un embedding usando ciertos campos de cada nodo

Pasos:
1. Construir un texto representativo de cada nodo a indexar que contiene un texto plano con el contenido de todas las propiedades de dicho nodo nodo.

    - EJEMPLO PERSONA:
        ```python
        "Persona: Juan, edad: 30, ciudad: Madrid"
        ```
    ----
    
    - PROBLEMA: En Neo4j los índices vectoriales solo aplican a nodos, así que hay que “materializar” cada relación como un nodo `:Doc` o `:Viaje`
        - EJEMPLO:
            ```python
            "Viaje: Luis con coche Ford Focus (2020). Detalle: Tuvo un accidente y no pudo viajar"
            ```


2. Guardar ese texto en una propiedad (`text`) del nodo 
3. Calcular y guardar el embedding en otra propiedad (`embedding`).
4. Crear el índice vectorial en Neo4j.
5. Usar Neo4jVector (LangChain) para recuperar por similitud.

Se pueden pasar varias propiedades en lugar de generar una nueva con todos los campos, pero se pierde coherencia, vería así: 
```python
- "Juan Madrid 30"
- "Ana Barcelona 25"
```

1. Construir un texto representativo de cada nodo a indexar que contiene un texto plano con el contenido de todas las propiedades de dicho nodo nodo.
2. Guardar ese texto en una propiedad (`text`) del nodo 

In [41]:
# personas
graph.query("""
MATCH (p:Persona)
SET p.text = 'Persona: ' + p.nombre +
             ', edad: ' + toString(p.edad) +
             ', ciudad: ' + coalesce(p.ciudad, '')
RETURN p.nombre, p.text;
""")

# coches
graph.query("""
MATCH (c:Coche)
SET c.text = 'Coche: ' + c.marca +
             ' ' + c.modelo +
             ', año: ' + toString(c.anio)
RETURN c.marca, c.modelo, c.text;
""")


[{'c.marca': 'Toyota',
  'c.modelo': 'Corolla',
  'c.text': 'Coche: Toyota Corolla, año: 2018'},
 {'c.marca': 'Ford',
  'c.modelo': 'Focus',
  'c.text': 'Coche: Ford Focus, año: 2020'},
 {'c.marca': 'Tesla',
  'c.modelo': 'Model 3',
  'c.text': 'Coche: Tesla Model 3, año: 2022'}]

- Tener en cuenta que para las relaciones hay que materializarlas en un nodo relacion

In [42]:
graph.query("""
MATCH (p:Persona)-[v:VIAJO_EN]->(c:Coche)
MERGE (d:Viaje {id: id(v)})   // crea un nodo Viaje por cada relación
SET d.text = 'Viaje: ' + p.nombre +
             ' con coche ' + c.marca + ' ' + c.modelo + 
             ' (' + toString(c.anio) + '). ' +
             'Detalle: ' + coalesce(v.descripcion_relacion, '')
MERGE (d)-[:DE_PERSONA]->(p)
MERGE (d)-[:EN_COCHE]->(c)
RETURN d.text;

""")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 21, offset: 63} for query: "\nMATCH (p:Persona)-[v:VIAJO_EN]->(c:Coche)\nMERGE (d:Viaje {id: id(v)})   // crea un nodo Viaje por cada relación\nSET d.text = 'Viaje: ' + p.nombre +\n             ' con coche ' + c.marca + ' ' + c.modelo + \n             ' (' + toString(c.anio) + '). ' +\n             'Detalle: ' + coalesce(v.descripcion_relacion, '')\nMERGE (d)-[:DE_PERSONA]->(p)\nMERGE (d)-[:EN_COCHE]->(c)\nRETURN d.text;\n\n"


[{'d.text': 'Viaje: Juan con coche Toyota Corolla (2018). Detalle: Fue su primer coche'},
 {'d.text': 'Viaje: Ana con coche Tesla Model 3 (2022). Detalle: Viajó con su madre para ver el nuevo modelo'},
 {'d.text': 'Viaje: Luis con coche Ford Focus (2020). Detalle: Tuvo un accidente y no pudo viajar'}]

In [40]:
print(graph.schema)

Node properties:
Persona {id: STRING, edad: INTEGER, ciudad: STRING, nombre: STRING}
Coche {id: STRING, anio: INTEGER, marca: STRING, modelo: STRING}
Relationship properties:
VIAJO_EN {descripcion_relacion: STRING}
The relationships:
(:Persona)-[:VIAJO_EN]->(:Coche)


3. Calcular y guardar el embedding en otra propiedad (`embedding`).
4. Crear el índice vectorial en Neo4j.
5. Usar Neo4jVector (LangChain) para recuperar por similitud.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector


EMBEDDINGS_MODEL = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Neo4jVector.from_existing_graph(
    url="bolt://localhost:50001", 
    username="neo4j", 
    password="frandovi", 
    database = "prueba-db",
    embedding=EMBEDDINGS_MODEL, #modelo para calcular embeddings
    node_label="Viaje", #nodos sobre los que se construye el indica vectorial
    text_node_properties=["text"], #propiedad que contiene el texto sobre el que se calcula el embedding
    embedding_node_property="embedding", #propiedad en la que se almacenan los embeddings
)

In [73]:
from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

llm = ChatOpenAI(temperature=0, model="gpt-4o")
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever, 
    chain_type="stuff",
    verbose=True,
    return_source_documents=True
)

In [75]:
res = qa.invoke({"query": "¿Quién tiene un tesla?"})
print(res["result"])
print("----- Fuentes -----")
for doc in res["source_documents"]:
    print(doc.page_content, doc.metadata)




> Entering new RetrievalQA chain...

> Finished chain.
Ana tiene un Tesla Model 3 (2022).
----- Fuentes -----

text: Viaje: Ana con coche Tesla Model 3 (2022). Detalle: Viajó con su madre para ver el nuevo modelo {}

text: Viaje: Juan con coche Toyota Corolla (2018). Detalle: Fue su primer coche {}


## Devolver nodos más relevantes de la BD. 

In [71]:
# solo textos/nodos (top-k)
docs = vectorstore.similarity_search("accidente en un viaje", k=2)
for d in docs:
    print(d.page_content, d.metadata)


text: Viaje: Luis con coche Ford Focus (2020). Detalle: Tuvo un accidente y no pudo viajar {}

text: Viaje: Juan con coche Toyota Corolla (2018). Detalle: Fue su primer coche {}


In [80]:
# con puntuación de similitud
docs_scores = vectorstore.similarity_search_with_score("corola", k=1)
for d, score in docs_scores:
    print(score, d.page_content, d.metadata)


0.6142463684082031 
text: Viaje: Juan con coche Toyota Corolla (2018). Detalle: Fue su primer coche {}


# Crear agente
Pasos
1. Definir modelo para el tool calling
2. Definir tolls
3. Definir PromptTemplate(System y Human)
4. Definir Agent
5. Definir AgentExecutor
6. Ejecutar AgentExecutor

In [92]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(dotenv_path=Path.cwd().parents[1] / ".env")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", None)
print(TAVILY_API_KEY)

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


In [100]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_tavily import TavilySearch
# 1. Definir modelo para el tool calling
llm = ChatOpenAI(model="gpt-4o", temperature=0)

#2. Definir tolls
tavily_tool = TavilySearch(
    include_answer=True,
    include_raw_content=False,
    max_results=5,
)

tools = [tavily_tool]

# 3. Definir PromptTemplate(System y Human)
system = (
    "Eres un agente útil. "
    "Cuando la pregunta requiera información actual o fuentes, usa la tool de búsqueda web. "
    "Cuando se pida información del grafo, razona y luego usa la tool neo4j_query con consultas READ (MATCH/RETURN). "
    "Devuelve respuestas concisas y con pasos si ejecutaste tools."
)
human = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder(variable_name = "chat_history"), #hueco en el prompt donde se inserta el historial de conversación  
    ("human", human),
    MessagesPlaceholder(variable_name = "agent_scratchpad"), #hueco donde el agente guarda su razonamiento intermedio (tools...)

])

# 4) Construir el runnable del agente
agent_runnable = create_tool_calling_agent(
    llm, 
    tools, 
    prompt,
    #handle_parsing_errors=True #si no quieres que responda sin tool
    )

# 5) Ejecutable final
agent = AgentExecutor(agent=agent_runnable, tools=tools, verbose=True)

In [134]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [158]:
from langchain_redis import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_redis import RedisChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
import asyncio
from langchain_core.messages import BaseMessage
class RedisSummaryMemory:
    def __init__(self, session_id, redis_url, llm, summary_every=4, ttl=60*60*24*7):


        # clase que permite conectarse a una BD de redis para gestionar la memoria del chat
        self.redis_history = RedisChatMessageHistory(session_id=session_id, redis_url=redis_url, ttl=ttl)
        self.llm = llm
        self.summary_every = summary_every
    
    @property
    def messages(self):
        # Retorna mensajes de forma síncrona o asíncrona según tu entorno
        # Si RedisChatMessageHistory.messages es async, hazlo async también
        return self.redis_history.messages


    async def add_user_message(self, content):
        await self.redis_history.add_user_message(content)
        await self._maybe_summarize()

    async def add_ai_message(self, content):
        await self.redis_history.add_ai_message(content)
        await self._maybe_summarize()

    async def _maybe_summarize(self):
        messages = self.redis_history.messages
        if len(messages) >= self.summary_every:
            # Construir texto para resumen
            conversation_text = "\n".join([f"{m.type}: {m.content}" for m in messages])
            prompt = f"Resume brevemente esta conversación:\n{conversation_text}\nResumen:"
            response = await self.llm.agenerate([HumanMessage(content=prompt)])
            summary = response.generations[[0]].text
            # Limpiar mensajes antiguos y guardar resumen como mensaje AI
            await self.redis_history.clear()
            await self.redis_history.add_ai_message(f"Resumen de la conversación: {summary}")
    
    async def aadd_messages(self, messages: list[BaseMessage]):
        for message in messages:
            if message.type == "human":
                await self.add_user_message(message.content)
            elif message.type == "ai":
                await self.add_ai_message(message.content)
            else:
                # Maneja otros tipos si es necesario
                pass

    async def aget_messages(self):
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, lambda: self.redis_history.messages)



#conectar a una sesion especifica de la BD y obtener historial de mensajes,
#se borran a los 7 dias
# def get_history(session_id: str):
#     return RedisSummaryMemory(
#         session_id=session_id,
#         redis_url="redis://localhost:6380/0",
#         llm=llm,
#         summary_every=4,
#         ttl=60*60*24*7
#     )

def get_session_history(session_id: str):
    return RedisChatMessageHistory(session_id=session_id, redis_url="redis://localhost:6380/0", ttl=None)

agent_with_memory = RunnableWithMessageHistory(
    runnable = agent, 
    get_session_history = get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output",
)


In [ ]:
# # Pregunta que mezcla web + grafo
# q1 = "¿Silvia García fue importante en la cooperativa Mediterranea?"

# print(agent_with_memory.invoke(
#     {"input": q1},
#     {"configurable": {"session_id": "usuario-123"}}

#     )
# )

# q1 = "Durante que años estuvo activa?"
# async for ev in agent_with_memory.astream_events(
#     {"input": q3},
#     config={"configurable": {"session_id": "prueba_futbol"}}, 
#     version="v1"):
#     et = ev["event"]

#     # ---- Texto del modelo (token a token) ----
#     if et == "on_chat_model_stream":
#         delta = ev["data"]["chunk"].content
#         print(delta, end="", flush=True)


18:55:48 redisvl.index.index INFO   Index already exists, not overwriting.


> Entering new AgentExecutor chain...
18:55:49 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Invoking: `tavily_search` with `{'query': 'Silvia García importancia cooperativa Mediterranea'}`


{'query': 'Silvia García importancia cooperativa Mediterranea', 'follow_up_questions': None, 'answer': 'Silvia García is an art director at LA Mediterranea LTD. She has over 25 years of experience in human resources management. She is also recognized for her leadership in cooperative sustainability initiatives.', 'images': [], 'results': [{'url': 'https://es.linkedin.com/in/silvia-garcia-005a2910a', 'title': 'Silvia Garcia - Art director en LA MEDITERRANEA LTD. - LinkedIn', 'content': 'Silvia Garcia\nArt director en LA MEDITERRANEA LTD.\nValencian Community, Spain\n89 connections, 91 followers\n\n\nAbout\nN/A\n\n\nExperience\nArt director\n[LA MEDITERRANEA LTD.](https://uk.l

In [ ]:
q1 = "Quien es el mejor futbolista de la historia?"
chat_model_response_printed = False

async for ev in agent_with_memory.astream_events(
    {"input": q1},
    config={"configurable": {"session_id": "prueba_futbol"}}, 
    version="v1"
):
    if ev["event"] == "on_prompt_start":
        print("---PROMPT ACTUAL---")
        print(ev["data"]["input"]["input"])
        
        print("---CHAT HISTORY---")
        print(ev["data"]["input"]["chat_history"])
    
    if ev["event"] == "on_chat_model_stream":
        if not chat_model_response_printed:
            print("---CHAT MODEL RESPONSE---")
            chat_model_response_printed = True
        
        delta = ev["data"]["chunk"].content
        print(delta, end="", flush=True)

11:06:44 redisvl.index.index INFO   Index already exists, not overwriting.




> Entering new None chain...
---PROMPT ACTUAL---
Quien es el mejor futbolista de la historia?
---CHAT HISTORY---
[]
11:06:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
La pregunta sobre quién es el mejor futbolista de la historia es subjetiva y depende de las opiniones personales de los aficionados al fútbol. Sin embargo, algunos de los nombres más mencionados en este debate incluyen a:

1. **Pelé**: Considerado uno de los más grandes de todos los tiempos, Pelé ganó tres Copas del Mundo con Brasil (1958, 1962, 1970) y es conocido por su habilidad, técnica y capacidad goleadora.

2. **Diego Maradona**: Famoso por su actuación en la Copa del Mundo de 1986, donde llevó a Argentina al título, Maradona es recordado por su increíble talento y momentos icónicos en el campo.

3. **Lionel Messi**: Con múltiples Balones de Oro y una carrera llena de récords y títulos con el FC Barcelona y la selección argentina, Messi es considerado por muchos 

In [198]:
q2 = "Cuantos balones de oro ha ganado?"
chat_model_response_printed = False

async for ev in agent_with_memory.astream_events(
    {"input": q2},
    config={"configurable": {"session_id": "prueba_futbol"}}, 
    version="v1"
):
    if ev["event"] == "on_prompt_start":
        print("---PROMPT ACTUAL---")
        print(ev["data"]["input"]["input"])
        
        print("---CHAT HISTORY---")
        print(ev["data"]["input"]["chat_history"])
    
    if ev["event"] == "on_chat_model_stream":
        if not chat_model_response_printed:
            print("---CHAT MODEL RESPONSE---")
            chat_model_response_printed = True
        
        delta = ev["data"]["chunk"].content
        print(delta, end="", flush=True)

11:12:43 redisvl.index.index INFO   Index already exists, not overwriting.


> Entering new None chain...
---PROMPT ACTUAL---
Cuantos balones de oro ha ganado?
---CHAT HISTORY---
[HumanMessage(content='Quien es el mejor futbolista de la historia?', additional_kwargs={}, response_metadata={}), AIMessage(content='La pregunta sobre quién es el mejor futbolista de la historia es subjetiva y depende de las opiniones personales de los aficionados al fútbol. Sin embargo, algunos de los nombres más mencionados en este debate incluyen a:\n\n1. **Pelé**: Considerado uno de los más grandes de todos los tiempos, Pelé ganó tres Copas del Mundo con Brasil (1958, 1962, 1970) y es conocido por su habilidad, técnica y capacidad goleadora.\n\n2. **Diego Maradona**: Famoso por su actuación en la Copa del Mundo de 1986, donde llevó a Argentina al título, Maradona es recordado por su increíble talento y momentos icónicos en el campo.\n\n3. **Lionel Messi**: Con múltiples Balones de Oro y una carrera llena 

In [201]:
history = get_session_history("prueba_futbol")

11:53:09 redisvl.index.index INFO   Index already exists, not overwriting.


In [202]:
history.messages

[HumanMessage(content='Quien es el mejor futbolista de la historia?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='La pregunta sobre quién es el mejor futbolista de la historia es subjetiva y depende de las opiniones personales de los aficionados al fútbol. Sin embargo, algunos de los nombres más mencionados en este debate incluyen a:\n\n1. **Pelé**: Considerado uno de los más grandes de todos los tiempos, Pelé ganó tres Copas del Mundo con Brasil (1958, 1962, 1970) y es conocido por su habilidad, técnica y capacidad goleadora.\n\n2. **Diego Maradona**: Famoso por su actuación en la Copa del Mundo de 1986, donde llevó a Argentina al título, Maradona es recordado por su increíble talento y momentos icónicos en el campo.\n\n3. **Lionel Messi**: Con múltiples Balones de Oro y una carrera llena de récords y títulos con el FC Barcelona y la selección argentina, Messi es considerado por muchos como el mejor futbolista de la era moderna.\n\n4. **Cristiano Ronaldo**: Con un

In [203]:
from langchain_core.output_parsers import StrOutputParser

llm_summarizer = ChatOpenAI(model="gpt-4o", temperature=0)

summary_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Resume la conversación de forma objetiva y comprimida, en español. "
     "Conserva: hechos, decisiones, tareas pendientes, fuentes citadas, parámetros importantes. "
     "No repitas texto literal salvo nombres/IDs. Limita a ~20 palabras."),
    MessagesPlaceholder("conversation"),
    ("human", "Crea un resumen útil para continuar la charla sin perder contexto.")
])
summary_chain = summary_prompt | llm_summarizer | StrOutputParser()



/var/folders/l6/l10s0t8n3cd4ghd9mjhdlltw0000gn/T/ipykernel_13289/4133568506.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_summarizer = ChatOpenAI(model="gpt-4o", temperature=0)


In [204]:
from langchain_core.messages import SystemMessage, BaseMessage

def summarize_into_system_message(messages: List[BaseMessage]) -> SystemMessage:
    summary_text = summary_chain.invoke({"conversation": messages})
    return SystemMessage(content=f"Resumen hasta ahora: {summary_text}")



In [205]:
msgs = history.messages
summary_msg = summarize_into_system_message(msgs)
history.clear()
history.add_message(summary_msg)


11:56:54 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [206]:
history.messages

[SystemMessage(content='Resumen hasta ahora: La conversación se centra en la cantidad de Balones de Oro ganados por Lionel Messi, quien ha obtenido el premio ocho veces, incluyendo en 2023.', additional_kwargs={}, response_metadata={})]